# Attendance Record

#### Stage 1: Import relevant modules

In [1]:
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import ipynb.fs
import ipywidgets as widgets

# import user defined module functions 
from .defs.CW_Preprocessing import PreprocessModuleRegister
from .defs.CW_Preprocessing import ModuleRegisterDB

In [2]:
# user inputs
folder = "cop504cwdata"
file = "22COA111ModuleRegister"
database = "CWDatabase.db"
module = "COA111" # (get user input of module + whether to concat module session i.e., session or attendance?) which will form 'table'
table = "COA111Sessions" # or COA111 
student_id = "0" 

pp = PreprocessModuleRegister(folder, file)
df_sessions, df_mod_atten = pp.get_module_attendance_csv()
df_clean_mod_atten = pp.clean_module_attendance(df_mod_atten)
 
modb = ModuleRegisterDB(
    database_name= database, 
    table_name= table, 
    df_module_data= df_sessions # or df_clean_mod_atten
)
modb.connect_to_db()
modb.write_df_to_db() # it also replaces table if it already exists
mod_sessions, mod_atten = modb.get_module_details_from_db()
mod_atten
mod_sessions

Dataframe has been successfully written to SQL!


,1,2,3,5,6,8,9,11,12,14
0,1.0,1.0,0.0,1.0,1,1,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,0.0,0,1,0.0,1.0,0.0,0.0
3,NaN,NaN,1.0,1.0,0,1,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0,1,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
194,0.0,0.0,1.0,1.0,0,1,1.0,1.0,1.0,1.0
195,1.0,0.0,1.0,1.0,1,1,1.0,1.0,1.0,0.0
196,1.0,1.0,1.0,1.0,0,1,1.0,1.0,1.0,1.0
197,1.0,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0


In [ ]:
class StudentAttendance:
    def __init__(self, database_name, student_id, module):
        self.db = database_name
        self.id = student_id
        self.mod = module

    def get_weekly_student_attendance(self): # tranpose after importing atten_db



    #==============================================================================

    # def add_student_attendance_to_db(): 

    # def add_session_to_db():

    # def remove_student_attendance():

    # def remove_session():


stu_att = StudentAttendance(
    database_name= database, 
    student_id= table, 
    module= df_sessions, # df_clean_mod_atten
    conn_status= conn
)
df_sessions = stu_att.get_sessions_from_db()

4. Get session and student attendance from database.
5. Add a new session to the database.
6. Add new student attendance to the database.

In [ ]:
# folder = "cop504cwdata"
# file = "22COA111ModuleRegister"

# pp = PreprocessModuleRegister(folder, file)
# df_sessions, df_mod_atten = pp.get_module_attendance_csv()
# df_clean_mod_atten = pp.clean_module_attendance(df_mod_atten)
# df_clean_mod_atten
# df_sessions

# database = "CWDatabase.db"
# table = "COA111Sessions" 
# module = ModuleRegisterDB(
#     database_name= database, 
#     table_name= table, 
#     df_module_data= df_sessions
# )
# module.connect_to_db()
# module.write_df_to_db() # it also replaces table if it already exists
# module.get_register_details_from_db()

#### Stage 2: Get relevant data from CWDatabase.db and format it to a list.

In [ ]:
class ReadSQL: # class Student (), class Sessions?
    def __init__(self, database):
        self.database = database
        self.conn = sqlite3.connect(self.database)
        self.cur = self.conn.cursor()

    def queryColumnsToDataframe(self, studentID, modules, tables):
        dfs = []
        for i in range(len(modules)):
            sub_query1 = 'SELECT Week, ROUND(CAST(SUM(' + studentID + ')'
            sub_query2 = ' AS REAL)/COUNT(Week) * 100, 2) AS ' + modules[i]
            sub_query3 = ' FROM ' + tables[i][0] + ' a INNER JOIN ' + tables[i][1] + ' b'
            sub_query4 = ' ON a.Session_Number = b.Session_Number' + ' GROUP BY Week'
            query = sub_query1 + sub_query2 + sub_query3 + sub_query4
            # print(query)
            try:
                df = pd.read_sql_query(query, self.conn)
                dfs.append(df)
            except Exception as Error:
                print('This ID does not exist in the database please try again.')
        df = pd.concat(dfs, join = 'outer', axis = 1, sort = False)
        # print(dfs)
        return df

    def formatDataframe(self, df):
        df.drop('Week', axis = 1, inplace = True) # drop duplicate week columns
        weekNum = df.index + 1
        df.insert(loc = 0, column = 'WeekNo', value = weekNum) # insert new week column at the start
        df.rename(
            columns = {
                'COA111':'COA111 (%)', 
                'COA122':'COA122 (%)'
            },
            inplace = True
        )
        return df

    def closeDatabase(self):
        self.conn.close() 
        print('\nThe sqlite connection is now closed.')

# Testing

### Stage 1: Given student id return the weekly attendance of the student

In [ ]:
sid = str(input('Please enter a student ID i.e., \'Student123\': '))
sid = sid.strip()

test = ReadSQL(database='CWDatabase.db')
modules = ['COA111', 'COA122']
sessions = [['COA111Sessions', 'COA111Transposed'], ['COA122Sessions', 'COA122Transposed']]

df = test.queryColumnsToDataframe(sid, modules, sessions)
weeklyAttendance = test.formatDataframe(df)
test.closeDatabase()

### Stage 2: Plot a bar chart of the student attendance

In [ ]:
week = np.array(weeklyAttendance['WeekNo'])

plt.bar(week- 0.2, weeklyAttendance['COA111 (%)'], width = 0.4, label = 'COA111') # w/o specifying colour
plt.bar(week + 0.2, weeklyAttendance['COA122 (%)'], width = 0.4, label = 'COA122')
plt.xlabel('Week Number')
plt.ylabel('Attendance')

# show the legend, grid and plot
plt.legend(loc = 'upper right')
plt.grid()

print('The weekly attendance record of Student ID %s:'%sid[7:])
plt.show()


In [ ]:
weeklyAttendance

#### Stage 4: Display list and graph to user. 

In [ ]:
# 1.3 https://stackoverflow.com/questions/28075699/coloring-cells-in-pandas